<a href="https://colab.research.google.com/github/jmand626/PyTorchMLEngine-Transfer-Learning/blob/main/PyTransferLearningPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This is a setup cell that should be reran at the begining of every time I reopen this notebook
import os
import sys
import zipfile
import requests
from pathlib import Path

# 1️⃣ Mount Google Drive (if using for storage)
use_gdrive = False  # Set to True if dataset is stored in Google Drive
if use_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

# 2️⃣ Clone your GitHub repo if it's not already present
repo_url = "https://github.com/jmand626/PyTorchMLEngine-Custom-Dataset-Project.git"
repo_name = "PyTorchMLEngine-Custom-Dataset-Project"

if not os.path.exists(repo_name):
    print(f"Cloning {repo_url}...")
    !git clone {repo_url}
else:
    print(f"Repository {repo_name} already exists.")

# 3️⃣ Change to repo directory
os.chdir(repo_name)

# 4️⃣ Add project files to sys.path so imports work
sys.path.append(os.getcwd())
print("Project directory added to sys.path")

# 5️⃣ Ensure necessary dependencies are installed
try:
    import torchinfo
except ImportError:
    print("Installing torchinfo...")
    !pip install -q torchinfo

# 6️⃣ Download FGVC Aircraft dataset if missing
data_path = Path("data/")
dataset_url = "https://www.robots.ox.ac.uk/~vgg/data/fgvc-aircraft/archives/fgvc-aircraft-2013b.tar.gz"
dataset_tar = data_path / "fgvc-aircraft-2013b.tar.gz"
dataset_folder = data_path / "fgvc-aircraft-2013b"

if dataset_folder.exists():
    print("Dataset already exists.")
else:
    print("Downloading FGVC Aircraft dataset...")
    data_path.mkdir(parents=True, exist_ok=True)
    response = requests.get(dataset_url, stream=True)
    with open(dataset_tar, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print("Extracting dataset...")
    !tar -xzf {dataset_tar} -C {data_path}
    os.remove(dataset_tar)
    print("Dataset extraction complete.")


In [3]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
torch version: 2.6.0+cu124
torchvision version: 0.21.0+cu124


In [4]:
!git clone https://github.com/jmand626/PyTorchMLEngine-Custom-Dataset-Project.git

fatal: destination path 'PyTorchMLEngine-Custom-Dataset-Project' already exists and is not an empty directory.


In [6]:
%cd PyTorchMLEngine-Custom-Dataset-Project
!ls

/content/PyTorchMLEngine-Custom-Dataset-Project
computer_vision_test_main.py  firsttry_model.py  README.md
create_custom_dataset.py      model_backbone.py  setup_dataholders.py


In [7]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [15]:
import os
import tarfile
import requests
from pathlib import Path

# Define the path to the data folder
data_path = Path("data/")
image_path = data_path / "fgvc_aircraft"

# Create the image directory if it doesn't exist
if not image_path.is_dir():
    print(f"Creating directory at {image_path}...")
    image_path.mkdir(parents=True, exist_ok=True)
else:
    print(f"Directory {image_path} already exists.")

# FGVC Aircraft dataset URL
dataset_url = "https://www.robots.ox.ac.uk/~vgg/data/fgvc-aircraft/archives/fgvc-aircraft-2013b.tar.gz"
dataset_path = data_path / "fgvc-aircraft-2013b.tar.gz"

# Download the dataset if it's not already present
if not dataset_path.is_file():
    print(f"Downloading FGVC Aircraft dataset from {dataset_url}...")
    response = requests.get(dataset_url, stream=True)
    with open(dataset_path, "wb") as file:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)
    print("Download complete.")
else:
    print(f"Dataset already downloaded at {dataset_path}.")

# Extract the dataset
if not any(image_path.iterdir()):
    print(f"Extracting {dataset_path} to {image_path}...")
    with tarfile.open(dataset_path, "r:gz") as tar:
        tar.extractall(path=image_path)
    print("Extraction complete.")
else:
    print(f"Dataset already extracted at {image_path}.")

# Verify extraction
print(f"Contents of {image_path}:")
print(list(image_path.iterdir())[:10])  # Display the first 10 items


Directory data/fgvc_aircraft already exists.
Download complete.
Extracting data/fgvc-aircraft-2013b.tar.gz to data/fgvc_aircraft...
Extraction complete.
Contents of data/fgvc_aircraft:
[PosixPath('data/fgvc_aircraft/fgvc-aircraft-2013b')]


Now hopefully we can continously use the previous setup code whenever we want to use this dataset again.

In [16]:
train_dir = image_path / "train"
test_dir = image_path / "test"

Now we continue on to creating our datasets and dataloaders. An important issue is that we have to ensure that the data that we feed into our pretrained model must be formatted in the same way as the data inputted when training the model (helps performance immeasurably). There is a certain way that all models from torchvision.models require, and we will do that.

In [18]:
import torchvision.transforms as transforms

manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)), # 1. Reshape all images to 224x224 (though some models may require different sizes)
    transforms.ToTensor(), # 2. Turn image values to between 0 & 1
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # 3. A mean of [0.485, 0.456, 0.406] (across each colour channel)
                         std=[0.229, 0.224, 0.225]) # 4. A standard deviation of [0.229, 0.224, 0.225] (across each colour channel),
])

**ISSUES BELOW**

In [30]:
# Problems with importing files, so I had to manually add files to syspath
import sys
import os
import importlib

# Add the current working directory to sys.path
sys.path.append(os.getcwd())

# List of Python files you want to import
modules = [
    "computer_vision_test_main",
    "create_custom_dataset",
    "firsttry_model",
    "model_backbone",
    "setup_dataholders"
]

# Import each module and reload to avoid caching issues
for module in modules:
    try:
        imported_module = __import__(module)
        importlib.reload(imported_module)  # Reload in case it was modified
        print(f"Successfully imported {module}")
    except Exception as e:
        print(f"Error importing {module}: {e}")


Error importing computer_vision_test_main: No module named 'FGVC_Aircraft'


100%|██████████| 2.75G/2.75G [02:44<00:00, 16.8MB/s]


Creating image split for: train...
Error importing create_custom_dataset: [Errno 2] No such file or directory: '../data/fgvc-aircraft-2013b/data/images/fgvc-aircraft-2013b/data/images_variant_train.txt'
Successfully imported firsttry_model
Successfully imported model_backbone
Successfully imported setup_dataholders


In [32]:
import os

dataset_path = "data/fgvc-aircraft-2013b"
if os.path.exists(dataset_path):
    print("✅ Dataset folder exists.")
    print("🗂 Contents:", os.listdir(dataset_path))
else:
    print("❌ Dataset folder is missing.")


❌ Dataset folder is missing.
